# Libraries

In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager

## Make the search in Linkdin be for a given job and country

In [13]:
job_name = "Data Analyst"
country_name = "United States"

job_url =""
for item in job_name.split(" "):
    if item != job_name.split(" ")[-1]:
        job_url = job_url + item + "%20"
    else:
        job_url = job_url + item

country_url =""
for item in country_name.split(" "):
    if item != country_name.split(" ")[-1]:
        country_url = country_url + item + "%20"
    else:
        country_url = country_url + item

url = "https://www.linkedin.com/jobs/search?keywords={0}&location={1}&geoId=103644278&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0"
url = url.format(job_url,country_url)  # assign the formatted URL to url variable

In [50]:
# Creating a webdriver instance
driver = webdriver.Chrome(ChromeDriverManager().install())

# Opening the url we have just defined in our browser
driver.get(url)

## Gets a list of the first couple of entries without clicking on them

In [15]:
#We get a list containing all jobs that we have found.
job_lists = driver.find_element(By.CLASS_NAME,"jobs-search__results-list")
jobs = job_lists.find_elements(By.TAG_NAME,"li") # return a list

#We declare void list to keep track of all obtaind data.
job_title_list = []
company_name_list = []
location_list = []
date_list = []
job_link_list = []

#We loof over every job and obtain all the wanted info.
for job in jobs:
    #job_title
    job_title = job.find_element(By.CSS_SELECTOR,"h3").get_attribute("innerText")
    job_title_list.append(job_title)
    
    #company_name
    company_name = job.find_element(By.CSS_SELECTOR,"h4").get_attribute("innerText")
    company_name_list.append(company_name)
    
    #location
    location = job.find_element(By.CSS_SELECTOR,"div>div>span").get_attribute("innerText")
    location_list.append(location)
    
    #date
    date = job.find_element(By.CSS_SELECTOR,"div>div>time").get_attribute("datetime")
    date_list.append(date)
    
    #job_link
    job_link = job.find_element(By.CSS_SELECTOR,"a").get_attribute("href")
    job_link_list.append(job_link)

#Create a dictionary to store the lists
data_dict = {"Job Title": job_title_list,
             "Company Name": company_name_list,
             "Location": location_list,
             "Date": date_list,
             "Job Link": job_link_list}

#Create a pandas DataFrame from the dictionary
df = pd.DataFrame.from_dict(data_dict)

df

,Job Title,Company Name,Location,Date,Job Link
0,Data Analyst,MagnaCare,"New York, NY",2023-03-02,https://www.linkedin.com/jobs/view/data-analys...
1,Data Analyst,Wipro,"Texas, United States",2023-03-31,https://www.linkedin.com/jobs/view/data-analys...
2,Data Analyst,"Staffing Opportunities Solutions, Inc.","New York, NY",2023-03-07,https://www.linkedin.com/jobs/view/data-analys...
3,Data Analyst,Human Services Administration Organization,"Pittsburgh, PA",2023-03-07,https://www.linkedin.com/jobs/view/data-analys...
4,Data Analyst,ContractRecruiter.com,"Suwanee, GA",2023-03-13,https://www.linkedin.com/jobs/view/data-analys...
5,Data Analyst,Fusion Alliance,"Cincinnati, OH",2023-03-11,https://www.linkedin.com/jobs/view/data-analys...
6,Data Analyst,Woodland Foods,"Waukegan, IL",2023-03-10,https://www.linkedin.com/jobs/view/data-analys...
7,Data Analyst,Zortech Solutions,United States,2023-03-31,https://www.linkedin.com/jobs/view/data-analys...
8,Data Analyst,La Clinica De Familia,"Las Cruces, NM",2023-03-11,https://www.linkedin.com/jobs/view/data-analys...
9,Data Analyst,"THOR Solutions, LLC","Alameda, CA",2023-03-31,https://www.linkedin.com/jobs/view/data-analys...


## Scrap the data from a specific job entry

### try to add seniority, employment type, job function, industries

In [43]:
# Find the first job listing and click on it
job_listings = driver.find_element(By.CLASS_NAME, "jobs-search__results-list")
first_job = job_listings.find_element(By.TAG_NAME, "li")
job_link = first_job.find_element(By.CSS_SELECTOR, "a")
job_link.click()

# Wait for the job details to load
time.sleep(3)

# Retrieve job details
job_title = driver.find_element(By.CLASS_NAME, "topcard__title").get_attribute("innerText")
company_name = driver.find_element(By.CLASS_NAME, "topcard__org-name-link").get_attribute("innerText")
job_description = driver.find_element(By.CLASS_NAME, "description__text").get_attribute("innerText")

# Try to retrieve the location element
try:
    location = driver.find_element(By.CLASS_NAME, "topcard__location").get_attribute("innerText")
except:
    location = None

# Print job details
print("Job Title: ", job_title)
print("Company Name: ", company_name)
print("Location: ", location)
print("Job Description: ", job_description)


Job Title:  Data Analyst (Seattle or US Remote)
Company Name:  Remitly 
Location:  None
Job Description:  Remitly is on a mission to transform the lives of immigrants and their families by providing the most trusted financial products and services on the planet. Since 2011, we have been tirelessly delivering on our promises to immigrants sending their hard earned money home. Today, we are reimagining international payments at scale and building new products to create deeper relationships with our customers and their loved ones across the globe. Join over 2,700 employees across 10 offices who are growing their careers while having a positive impact on people globally.

About The Role

As a Data Analyst on the Customer Success team, you will report to the Analytics Manager, Customer Success, and help Remitly serve our customers. You'll immerse yourself in the customer experience, identify sources of customer friction, and develop quantitative insights. You'll partner with program manager

## Make the process more automated and go for the first and second job entries

In [52]:
# Find all job listings and loop through them
job_listings = driver.find_elements(By.CLASS_NAME, "jobs-search__results-list li")
for i in range(len(job_listings)):
    # Find the job listing again to avoid stale element reference
    job_listings = driver.find_elements(By.CLASS_NAME, "jobs-search__results-list li")
    job = job_listings[i]

    # Click on the job listing
    job_link = job.find_element(By.CSS_SELECTOR, "a")
    job_link.click()

    # Wait for the job details to load
    time.sleep(3)

    # Retrieve job details
    job_title = driver.find_element(By.CLASS_NAME, "topcard__title").get_attribute("innerText")
    company_name = driver.find_element(By.CLASS_NAME, "topcard__org-name-link").get_attribute("innerText")
    job_description = driver.find_element(By.CLASS_NAME, "description__text").get_attribute("innerText")

    # Try to retrieve the location element
    try:
        location = driver.find_element(By.CLASS_NAME, "topcard__location").get_attribute("innerText")
    except:
        location = None

    # Print job details
    print("Job Title: ", job_title)
    print("Company Name: ", company_name)
    print("Location: ", location)
    print("Job Description: ", job_description)

    # Go back to the initial URL to look for the next job listing
    driver.back()

Job Title:  Data Analyst
Company Name:  MagnaCare 
Location:  None
Job Description:  About the Role



The Data Analytics team maintains the provider and member databases. The team performs analysis of healthcare provider and member data for specified clients/products, ensuring the integrity and accuracy of data, performing root cause analysis, reporting results and processing claim holds. The Data Analyst will exercise due diligence and take proactive measures when necessary while tracking all work in a reportable fashion.



Primary Responsibilities



Maintain integrity of provider data including network affiliations, provider demographics, and updating fee schedule and contract assignments for participating facilities.
Maintain integrity of member eligibility and demographics manually, as well as review and load electronic membership data received from various clients.
Work with the Contracting Department to identify and resolve issues with contracted groups, as well as database im

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".topcard__title"}
  (Session info: chrome=111.0.5563.147)
